In [ ]:
%matplotlib inline


[Learn the Basics](intro.html) ||
[Quickstart](quickstart_tutorial.html) ||
[Tensors](tensorqs_tutorial.html) ||
[Datasets & DataLoaders](data_tutorial.html) ||
[Transforms](transforms_tutorial.html) ||
**Build Model** ||
[Autograd](autogradqs_tutorial.html) ||
[Optimization](optimization_tutorial.html) ||
[Save & Load Model](saveloadrun_tutorial.html)

# Build the Neural Network

Neural networks comprise of layers/modules that perform operations on data.
The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace provides all the building blocks you need to
build your own neural network. Every module in PyTorch subclasses the [nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## Get Device for Training
We want to be able to train our model on a hardware accelerator like the GPU,
if it is available. Let's check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) is available, else we
continue to use the CPU.



In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [7]:
#study the function torch.nn.Flatten()
input = torch.randn(32, 2,1, 5, 5,3)
# With default parameters
# the default is to combine the rest dimensions except the first dimension
m = nn.Flatten()
output = m(input)
print(output.size())
# With non-default parameters
m = nn.Flatten(0, 2)
output = m(input)
print(output.size())
m = nn.Flatten(0, -1)
output = m(input)
print(output.size())

torch.Size([32, 150])
torch.Size([64, 5, 5, 3])
torch.Size([4800])


## Define the Class
We define our neural network by subclassing ``nn.Module``, and
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method.



In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print
its structure.



In [9]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
X = torch.rand(1, 28, 28, device=device)
print(X.shape)
#y0= torch.flatten(X)
#torch.flatten is different from torch.nn.Flatten()
#m = nn.Flatten()
#y0 = m(X)
y0 = nn.Flatten()(X)
print(y0.shape)
#print(y0)

torch.Size([1, 28, 28])
torch.Size([1, 784])


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some [background operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, and dim=1 corresponding to the individual values of each output.  .
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [16]:
X = torch.rand(1, 28, 28, device=device)
print(type(X))
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

<class 'torch.Tensor'>
Predicted class: tensor([3])


--------------




## Model Layers

Let's break down the layers in the FashionMNIST model. To illustrate it, we
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as
we pass it through the network.



In [26]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten
We initialize the [nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [27]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear
The [linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its stored weights and biases.




In [28]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use [nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html) between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [30]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")
print(hidden1.size())

Before ReLU: tensor([[0.0000, 0.1761, 0.0000, 0.4344, 0.1367, 0.0576, 0.0000, 0.0000, 0.0000,
         0.0349, 0.0000, 0.0000, 0.0000, 0.4045, 0.3916, 0.0000, 0.0000, 0.0000,
         0.5562, 0.0000],
        [0.0186, 0.1599, 0.0000, 0.0246, 0.3026, 0.0000, 0.0067, 0.0000, 0.0000,
         0.0885, 0.0000, 0.0000, 0.0000, 0.0239, 0.1412, 0.0000, 0.0000, 0.0000,
         0.7734, 0.0000],
        [0.0000, 0.0741, 0.0000, 0.2407, 0.1007, 0.0387, 0.0000, 0.0000, 0.0000,
         0.5171, 0.0000, 0.0000, 0.0000, 0.2923, 0.1480, 0.3606, 0.0000, 0.0000,
         0.9033, 0.0000]], grad_fn=<ReluBackward0>)


After ReLU: tensor([[0.0000, 0.1761, 0.0000, 0.4344, 0.1367, 0.0576, 0.0000, 0.0000, 0.0000,
         0.0349, 0.0000, 0.0000, 0.0000, 0.4045, 0.3916, 0.0000, 0.0000, 0.0000,
         0.5562, 0.0000],
        [0.0186, 0.1599, 0.0000, 0.0246, 0.3026, 0.0000, 0.0067, 0.0000, 0.0000,
         0.0885, 0.0000, 0.0000, 0.0000, 0.0239, 0.1412, 0.0000, 0.0000, 0.0000,
         0.7734, 0.0000],
       

### nn.Sequential
[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) is an ordered
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [32]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits.shape)

torch.Size([3, 10])


### nn.Softmax
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html) module. The logits are scaled to values
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along
which the values must sum to 1.



In [33]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab.shape)

torch.Size([3, 10])


## Model Parameters
Many layers inside a neural network are *parameterized*, i.e. have associated weights
and biases that are optimized during training. Subclassing ``nn.Module`` automatically
tracks all fields defined inside your model object, and makes all parameters
accessible using your model's ``parameters()`` or ``named_parameters()`` methods.

In this example, we iterate over each parameter, and print its size and a preview of its values.




In [34]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0251, -0.0225, -0.0291,  ..., -0.0333, -0.0305, -0.0202],
        [ 0.0015,  0.0071,  0.0158,  ..., -0.0078,  0.0048, -0.0339]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0068,  0.0320], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0066,  0.0319,  0.0301,  ..., -0.0057, -0.0223,  0.0435],
        [ 0.0202,  0.0206,  0.0056,  ..., -0.0276,  0.0382,  0.0273]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

--------------




## Further Reading
- [torch.nn API](https://pytorch.org/docs/stable/nn.html)

